<h1 align='center'>
DECODING WORKSHOP
</h1>


# Introduction

## Jupyter notebook usage

This notebook allows to run python code interactively. Each code cell can be edited and run separately.

### Run a cell
Click on the following cell to select it and press the `Run` button or use the shortcut `Shitf+Enter`

In [ ]:
# Click here and press Shift + Enter 
print('Hello World')

### Variables scopes
All variables and functions of the notebooks are shared. Be careful not to override already existing variables by mistake!

In [ ]:
# Here we declare a variable
tuto_string = 'Initial string'

In [ ]:
# Let's print it
print(tuto_string)

In [ ]:
# Then we override it
tuto_string = 'Overriden string'
print('now run the previous cell again to print the string')

**→ The order in which you run the cells matters**

## Python basics

Python is a free and open source programming language that aims at being readable and fast to program with.

A big strength of Python is its extensive standard library that provides convenient tools for usual tasks. It is also very easy to install or distribute modules with it. Many high quality modules have been created by the community, in particular we'll use a few targetting machine learning.

You can skip this part if you're already familiar with Python 👍

### Math operations

In [ ]:
# declaring variables
a = 2
b = 3.5

In [ ]:
# sum
a + b

In [ ]:
# multiplication
a * b

In [ ]:
# division
a / b

In [ ]:
# integer division
5 // 2

In [ ]:
# power (5^2)
5 ** 2

### Lists

In [ ]:
l = [1, 4.0, 5, 'hi', [0,1,2]]
print(l)

In [ ]:
# accessing the first element
l[0]

In [ ]:
# accessing the last element
l[-1]

In [ ]:
# you can access any element of a list in a list
l[-1][2]

In [ ]:
# accessing a slice
l[1:4]

In [ ]:
# many objects have a length that you can get with len()
len(l)

### For loops

In [ ]:
fruits = ['apple', 'banana', 'orange']

for fruit in fruits:
    print(fruit)

In [ ]:
for i in range(4):
    print(i)

In [ ]:
# let's find the fruit names with an 'a'

fruits = ['apple', 'banana', 'orange', 'kiwi']
has_an_a = []
   
for fruit in fruits:
    if 'a' in fruit:
        has_an_a.append(fruit) # we can add an element to a list with list.append(element)
   
print("List of fruits with an 'a':", has_an_a)


### Comparisons

In [ ]:
the_basket = ['apple', 'banana', 'lemon']

my_favorite = 'lemon'

for fruit in the_basket:
    if fruit == my_favorite:
        print(f"There's a {my_favorite} left in the basket!")
    else:
        print(f"I don't like {fruit}")



In [ ]:
5 < 4

In [ ]:
'python' != 5.2

In [ ]:
'True' == True

In [ ]:
4 <= 4

### Functions

In [ ]:
def addition(x,y):
    return x + y

res = addition(6,5)
print(res)

In [ ]:
# parameters can be optional if they have a default value

def print_passion(name='trains'):
    print(f'I like {name}')
    
print_passion()
print_passion(name='knitting')

In [ ]:
# you can return multiple values

def stats(list_of_numbers):
    return max(list_of_numbers), min(list_of_numbers) # min and max functions are built-in

l = [3,6,1,8,12]

maximum, minimum = stats(l)

print(f'{maximum}, {minimum}')

### Comprehensions

Due to it's nature, Python is rather slow for large for loops, in particular when adding elements to a list with append. 

It is common to use comprehensions instead for both conciseness and optimization:

In [ ]:
square_numbers = [i**2 for i in range(10)]

print(square_numbers)

In [ ]:
# list comprehensions support conditions

fruits = ['apple', 'banana', 'orange', 'kiwi']

has_an_a = [fruit for fruit in fruits if 'a' in fruit]

print(has_an_a)

# TODO: Import libraries

In [ ]:
# Import packages

import numpy as np # library providing efficient array manipulation
import sklearn # machine learning tools
import scipy
import matplotlib.pyplot as plt # matlab-like plot library
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

from librosa.util import frame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score, mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

from pyriemann.estimation import Covariances, XdawnCovariances
from pyriemann.classification import MDM


from scipy.signal import iirdesign, sosfilt, convolve, wiener

import tp # load python module written for this project

# Part 0 : Dataset

In this workshop, we'll decode finger flexion from ECoG recordings. The data includes the recordings of the flexion of the subjects 5 fingers.

More details here: https://www.bbci.de/competition/iv/desc_4.pdf

### Download dataset

Run the following cell to download the dataset. 

If it fails, please manually download and extract to `tp-decoding/data/bciciv/`:
1. the training data: https://www.bbci.de/competition/download/competition_iv/BCICIV_4_mat.zip
2. the evaluation data: https://www.bbci.de/competition/iv/results/ds4/true_labels.zip



In [ ]:
dataset_path = 'data/bciciv'
if not tp.dataset_exists(dataset_path):
    tp.download_dataset(dataset_path)
else:
    print(f'dataset already exists in {dataset_path}') 


### Load dataset

In [ ]:
dataset = tp.load_data(dataset_path)
fs = 1000 # sampling frequency of the dataset

### Explore dataset

In [ ]:
for subject in dataset:
    print(len(subject.train_ecog))

In [ ]:
print(len(dataset))

In [ ]:
subject_data = dataset[0] # first subject in dataset
print(subject_data.train_fingers)

In [ ]:
# the data is stored in numpy arrays, you can get their size by accessing their 'shape' property

subject_data.train_fingers.shape

In [ ]:
# get first sample for all fingers, indexing starts at 0 /!\

subject_data.train_fingers[0, :]

In [ ]:
# get last sample for all fingers

subject_data.train_fingers[-1, :]

In [ ]:
# get 100 samples from index 1200 to 1299 for the 5th finger

subject_data.train_fingers[1200:1300, 4]

### Plot the data with matplotlib
You can find the documentation for the `plot` function here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

For more information, you can check the [quick start guide](https://matplotlib.org/stable/users/explain/quick_start.html)

In [ ]:
n_fingers = subject_data.train_fingers.shape[1]

fig = plt.figure(figsize=(50,20))
axes = fig.subplots(nrows=n_fingers, ncols=1, sharex=True)

for i in range(n_fingers):
    axes[i].plot(subject_data.train_fingers[:,i])
    axes[i].set_ylabel(f'finger {i+1}')

_ = fig.suptitle('Finger trajectories (train set)')

# you can double clik on large plots to zoom in

In [ ]:
f_ratio = 40


(n_times_train, n_fingers) = subject_data.train_fingers.shape
n_channels = subject_data.train_ecog.shape[1]
n_times_train = n_times_train//f_ratio
threshold = 0.5
n_times_test = subject_data.test_fingers.shape[0]//f_ratio

win_size = 25
win = 1/win_size*np.ones([win_size, 1])
train_fingers_filtered = convolve(subject_data.train_fingers[::f_ratio], win, 'same')
train_fingers_filtered = np.concatenate((train_fingers_filtered, threshold*np.ones([n_times_train, 1])), axis=1)
y_train = np.argmax(train_fingers_filtered, axis=1)
cue_train = np.zeros([n_times_train, n_fingers+1])
for i in range(n_times_train):
    cue_train[i, y_train[i]] = 1

test_fingers_filtered = convolve(subject_data.test_fingers[::f_ratio], win, 'same')
test_fingers_filtered = np.concatenate((test_fingers_filtered, threshold*np.ones([n_times_test, 1])), axis=1)
y_test = np.argmax(test_fingers_filtered, axis=1)
cue_test = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_test[i, y_test[i]] = 1

# Save the full data for future use
y_train_full = y_train
y_test_full = y_test

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1, sharex=True)
for i in range(n_fingers):
    ax[0].plot(subject_data.train_fingers[::f_ratio, i])
    ax[1].plot(train_fingers_filtered[:, i])
    ax[2].plot(cue_train[:, i])
ax[1].plot([0, n_times_train], [threshold, threshold], color='k')
fig.suptitle('Finger trajectories (train set)')


fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1, sharex=True)
for i in range(n_fingers):
    ax[0].plot(subject_data.test_fingers[::f_ratio, i])
    ax[1].plot(test_fingers_filtered[:, i])
    ax[2].plot(cue_test[:, i])
ax[1].plot([0, n_times_test], [threshold, threshold], color='k')
_ = fig.suptitle('Finger trajectories (test set)')

# click on the figure to expand

In [ ]:
# Let's inspect the ECoG channels

# tp.plot_ecog(subject_data.train_ecog) # scroll on the figure to see all channels

In [ ]:
# Obviously there's something wrong, let's plot it again without problematic channels

# exclude_channels = [54] # list the channels you'd like to remove

# n_channels = subject_data.train_ecog.shape[1]
# channel_selection = [i for i in range(n_channels) if i not in exclude_channels]

# tp.plot_ecog(subject_data.train_ecog, channel_selection)

# Part 1 : Dicrete decoding

In this part we will be interested in decoding finger mouvement during tapping from ECOG data and in order to do so we will focus on start with feature extraction by extracting time-frequency features on every electrodes using short term fourier transfrom. 

After this we will get a first set of results that we will visualize using different metrics such as the confusion matrix, the accuracy and the balanced accuracy. 

Then we will have an example of the usefullness of a validation set in the case of lagged responses. 

Once the lag have been set, we will see that we can also get better results using spatial filters such as PCA.

## 1.1 Feature extraction

### Temporal filtering

In this part we will focus on the filtering of the data using classical filtering function such as `iirdesign` and `sosfilt` from `scipy.signal` (https://docs.scipy.org/doc/scipy/reference/signal.html). 
It is pretty common in ECOG to filter in several frequency bands and use all the resulting features for the decoding.

In [ ]:
# bands = [(1, 60), (60, 100), (100, 200)]
# bands = [(1, 10)]
# for i in range(1, 20):
#     bands.append((i*10, (i+1)*10))
filters = []

bands = [(1, 10), (10,30), (30,50), (70,200)]
n_filters = len(bands)

for i_band, band in enumerate(bands):
   # filters.append(iirdesign(band[0], band[1], 1, 60, analog=False, fs=fs, output='sos'))
    filters.append(iirdesign(band, (band[0]*0.9, band[1]*1.1), 2, 20, analog=False, fs=fs, output='sos'))

ecog_train = tp.common_average_reference(subject_data.train_ecog, axis=1)
ecog_test = tp.common_average_reference(subject_data.test_ecog, axis=1)


ecog_train = subject_data.train_ecog
ecog_test = subject_data.test_ecog


scaler = StandardScaler()
train_ecog_normalized = scaler.fit_transform(ecog_train)
test_ecog_normalized = scaler.transform(ecog_test)

train_ecog_filtered = []
test_ecog_filtered = []

for filt in filters:
    train_ecog_filtered.append(sosfilt(filt, train_ecog_normalized, axis=0))
    test_ecog_filtered.append(sosfilt(filt, test_ecog_normalized, axis=0))

fig = plt.figure(figsize=(50,15))
ax = fig.subplots(nrows=4, ncols=1)

ax[0].plot(train_ecog_filtered[0][:1000, 0])
ax[1].plot(train_ecog_filtered[1][:1000, 0])
ax[2].plot(train_ecog_filtered[2][:1000, 0])
ax[3].plot(train_ecog_filtered[3][:1000, 0])

ax[0].plot(train_ecog_normalized[:1000, 0])
ax[1].plot(train_ecog_normalized[:1000, 0])
ax[2].plot(train_ecog_normalized[:1000, 0])
ax[3].plot(train_ecog_normalized[:1000, 0])

fig.suptitle('Filtered data (train set)')

### Average power computation

Once the signal has been filtered, you can extract the average power of the signal in those frequency band using the `power` and `mean` function from `numpy`. Note that the formula for the average power of a centered signal $x$ is 

\begin{align}
P(x) = \frac{1}{N}\sum_{t}x^2(t),
\end{align}

with $N$ the number of elements in the window.  

In this part we will extract power in 1s of signal every 40ms. In order to compute the frame to be averaged you can use the fonction `frame` from `librosa.util`(https://librosa.org/doc/latest/generated/librosa.util.frame.html) and the padding function `pad` from `numpy` because for the first frames you can't take the 1s of previous signal as it doesn't exist, therefore you can replace it with zeros.  

For an easier buffering version, you can use the `tp.buffering` function that has been directly created for this hand on. 

In [ ]:
win_size = 1000
n_buffer = 1;

X_train_filtered = np.concatenate(train_ecog_filtered, axis=1)
print(X_train_filtered.shape)
X_train = tp.buffering_power(X_train_filtered, win_size, f_ratio, n_buffer)
X_train_full = X_train
print(X_train_full.shape)

X_test_filtered = np.concatenate(test_ecog_filtered, axis=1)
X_test = tp.buffering_power(X_test_filtered, win_size, f_ratio, n_buffer)
X_test_full = X_test

### Set if we use the idle or not

In order to make things easier you can chose to select only the fingers and not the idle task.

In [ ]:

use_idle_data = True
if not use_idle_data:
    X_train = X_train_full[y_train_full != 5]
    X_test = X_test_full[y_test_full != 5]
    y_train = y_train_full[y_train_full != 5]
    y_test = y_test_full[y_test_full != 5]
    n_times_train = y_train.shape[0]
    n_times_test = y_test.shape[0]
else:
    X_train = X_train_full
    X_test = X_test_full
    y_train = y_train_full
    y_test = y_test_full
    n_times_train = y_train.shape[0]
    n_times_test = y_test.shape[0]
    

In [ ]:
print(X_train.shape)
print(y_train.shape)

## 1.2 Classification

### The classifier

A classifier is an algorithm that aims to classify given inputs (here the power in given frequency bands) into outputs (here our 5 possible fingers + the idle class where the user does nothing). 

In this particular example we will use either the `LinearDiscriminantAnalysis` (imported here as `LDA`) from `sklearn.discriminant_analysis` or the `LinearSVC` from `sklearn.svm`. 
See their documentation here : 

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

In the sklearn API, all classes are using the `fit` function to train the algorithm and the `predict` function to get the predictions for the test set. 

### Fit the classifier

In [ ]:
estimator = LDA()

estimator.fit(X_train, y_train)

### Get the predicted labels

In [ ]:
y_pred = estimator.predict(X_test)
y_proba = estimator.predict_proba(X_test)

cue_pred = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_pred[i, y_pred[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)

cue_test = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_test[i, y_test[i]] = 1

for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

## 1.3 Confusion matrix and metrics

Once you get your predictions you usually want to evaluate the performance of your decoder. In order to do so, a lot of different metrics have been developped with time. Most of the are using what is called a confusion matrix in order to be computed. 

### The confusion matrix

The confusion matrix represents how well our classifier is performing. The rows are the true labels we are expected to obtain and the columns are the labels our classifier actually gives us. 

In order to plot the confusion matrix, you can use the `from_predictions` function from the `ConfusionMatrixDisplay` class of `sklearn.metrics`.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred) # , text_kw={'fontsize': 10}

### The accuracy

A good way to consider how well the classifer is performing using the confusion matrix is the accuracy. The accuracy is equal to the total number of well predicted labels divided by the total number of labels. In the confusion matrix it is equal to the sum of the diagonal divided by the sum of all the elements of the matrix. 

You can get the accuracy using the `accuracy_score` function from `sklearn.metrics`. 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

### The balanced accuracy

In some cases there are more labels from some classes than others, for example here the finger 2 and 5 are less represented than others so their accuracy accounts for less than the others when we compute the accuracy metric. A good way to consider every labels with the same weight is to compute the balanced accuracy. It gives the accuracy for each class reweighted so that every class accounts for as much as the others. 

It is computed using the `balanced_accuracy_score` function from `sklearn.metrics`.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn-metrics-balanced-accuracy-score

In [ ]:
for i in range(n_fingers+1):
    print("There are {} labels in test for class {}".format(np.sum(y_test==i), i+1))

plt.hist(y_test, bins=n_fingers+1, rwidth=0.5)


In [ ]:
acc = balanced_accuracy_score(y_test, y_pred)
print("Average score is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.4 The validation set

Sometimes algorithms and methods need to set a specific parameter to get better results. In order to do so, we usually use what we call a validation set. This set is usually a subdivision of the training set that works as a validation for the parameter we are trying to optimize. In the next example, we will try to find the best lag between the input and the output.

### Use the validation to compute a lag value

In [ ]:
lags = [i for i in range(-26, 26, 2)] 
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=13, shuffle=False)
lagged_accuracies = []
n_times_train_val = X_train_val.shape[0]
n_times_val = X_val.shape[0]
for i_lag, lag in enumerate(lags):
    X_train_lagged = X_train_val[max(-lag, 0):min(n_times_train_val-lag, n_times_train_val)]
    y_train_lagged = y_train_val[max(lag, 0):min(n_times_train_val+lag, n_times_train_val)]
    X_val_lagged = X_val[max(-lag, 0):min(n_times_val-lag, n_times_val)]
    y_val_lagged = y_val[max(lag, 0):min(n_times_val+lag, n_times_val)]
    
    estimator.fit(X_train_lagged, y_train_lagged)
    y_pred_val = estimator.predict(X_val_lagged)
    lagged_accuracies.append(accuracy_score(y_val_lagged, y_pred_val))
    print("Lag {}/{} ({:.2f}s) -> Accuracy = {:.2f}%".format(i_lag+1, len(lags), lag/25, lagged_accuracies[-1]*100))

### Compute the best lag

In [ ]:
plt.plot(np.linspace(-1, 1, len(lagged_accuracies)), lagged_accuracies)
best_lag = lags[np.argmax(lagged_accuracies)]
print("The best lag is {}. This lag correspond to {:.2f}s".format(best_lag, best_lag/25))

### Compute lagged X and y

In [ ]:
X_train_lagged = X_train[max(-best_lag, 0):min(n_times_train-best_lag, n_times_train)]
y_train_lagged = y_train[max(best_lag, 0):min(n_times_train+best_lag, n_times_train)]
n_times_train_lagged = n_times_train-abs(best_lag)

X_test_lagged = X_test[max(-best_lag, 0):min(n_times_test-best_lag, n_times_test)]
y_test_lagged = y_test[max(best_lag, 0):min(n_times_test+best_lag, n_times_test)]
n_times_test_lagged = n_times_test-abs(best_lag)

### Train the estimator and decode testing data with lagged estimates of X and y

In [ ]:
estimator.fit(X_train_lagged, y_train_lagged)
y_pred = estimator.predict(X_test_lagged)
y_proba = estimator.predict_proba(X_test_lagged)

cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

### Compute metrics 

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred)

acc = accuracy_score(y_test_lagged, y_pred)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.5 Feature selection and dimensionality reduction

In some cases in machine learning there are so much features that the model cannot correctly learn for the training data. In these cases, it is very common to compute dimensionnality reduction or spatial filters in order to keep only the features of interest for decoding. In this example we will use a Principal Components Analysis (the `PCA` class from `sklearn.decomposition`) and compute patterns for each finger in order to improve the results.

In `sklearn`, the classes that transform data without predicting are called transformers and use the functions `fit` and `transform` to process data. 

In [ ]:
X_train_pca = []
X_test_pca = []
for i in range(n_fingers):
    print("Computing PCA patterns for finger {}...".format(i+1))
    s_filt = PCA(60)
    s_filt.fit(X_train_lagged[y_train_lagged==i])
    X_train_pca.append(s_filt.transform(X_train_lagged))
    X_test_pca.append(s_filt.transform(X_test_lagged))

X_train_pca = np.concatenate(X_train_pca, axis=1)
X_test_pca = np.concatenate(X_test_pca, axis=1)

estimator.fit(X_train_pca, y_train_lagged)
y_pred_pca = estimator.predict(X_test_pca)
y_proba_pca = estimator.predict_proba(X_test_pca)

In [ ]:
cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred_pca[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba_pca[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred_pca)


acc = accuracy_score(y_test_lagged, y_pred_pca)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred_pca)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.6 Get a more stable output (optional)

On decoders that are to be used by humans, it is important to get a more stable output so that it doesn't radically change every 40ms for example. This does not always improve classification but it can improve the feeling of control for the user with the side effect of adding a bit of lag. 

In this way, we can get the output probability of the decoder using the `predict_proba` function and smooth it using a convolution with a window. 

In [ ]:
win_size = 25
win = np.expand_dims(scipy.signal.windows.hamming(win_size), 1)
y_proba_stable = convolve(y_proba_pca, win, 'same')
y_pred_stable = np.argmax(y_proba_stable, axis=1)

cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred_stable[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba_stable[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

### Get metrics

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred_stable)

acc = accuracy_score(y_test_lagged, y_pred_stable)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred_stable)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/6))

### Riemannian Geometry

In [ ]:
X_train_buffer.shape

In [ ]:
# X_train_filtered = np.concatenate(train_ecog_filtered, axis=1)
X_train_filtered = train_ecog_filtered[1]
win_size = 1000
n_channels = X_train_filtered.shape[1]

X_train_buffer = np.pad(X_train_filtered, [(win_size-f_ratio, 0), (0, 0)])
X_train_buffer = frame(X_train_buffer, frame_length=win_size, hop_length=f_ratio, axis=0)
n_trials = X_train_buffer.shape[0]
X_train_buffer = np.transpose(X_train_buffer.reshape(n_trials, win_size, n_channels), (0, 2, 1))


X_test_filtered = np.concatenate(test_ecog_filtered, axis=1)
X_test_filtered = test_ecog_filtered[1]
X_test_buffer = np.pad(X_test_filtered, [(win_size-f_ratio, 0), (0, 0)])
X_test_buffer = frame(X_test_buffer, frame_length=win_size, hop_length=f_ratio, axis=0)
n_trials_test = X_test_buffer.shape[0]
X_test_buffer = np.transpose(X_test_buffer.reshape(n_trials_test, win_size, n_channels), (0, 2, 1))


X_train_lagged = X_train_buffer[max(-best_lag, 0):min(n_times_train-best_lag, n_times_train)]
y_train_lagged = y_train[max(best_lag, 0):min(n_times_train+best_lag, n_times_train)]
n_times_train_lagged = n_times_train-abs(best_lag)

X_test_lagged = X_test_buffer[max(-best_lag, 0):min(n_times_test-best_lag, n_times_test)]
y_test_lagged = y_test[max(best_lag, 0):min(n_times_test+best_lag, n_times_test)]
n_times_test_lagged = n_times_test-abs(best_lag)

In [ ]:
# cov = XdawnCovariances(estimator="oas")

# X_train_cov = cov.fit_transform(X_train_buffer, y_train)
# X_test_cov = cov.transform(X_test_buffer)
# print(X_train_cov.shape)

In [ ]:
# estimator = MDM()
# estimator.fit(X_train_cov, y_train)

In [ ]:
# y_pred = estimator.predict(X_test_cov)
# y_proba = estimator.predict_proba(X_test_cov)

# cue_pred = np.zeros([n_times_test, n_fingers+1])
# for i in range(n_times_test):
#     cue_pred[i, y_pred[i]] = 1

# fig = plt.figure(figsize=(150,5))
# ax = fig.subplots(3, 1)

# cue_test = np.zeros([n_times_test, n_fingers+1])
# for i in range(n_times_test):
#     cue_test[i, y_test[i]] = 1

# for i in range(n_fingers):
#     ax[0].plot(cue_test[:, i])
#     ax[1].plot(y_proba[:, i])
#     ax[2].plot(cue_pred[:, i])
# fig.suptitle('Finger trajectories (test set)')


# acc = accuracy_score(y_test, y_pred)
# print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

# acc = balanced_accuracy_score(y_test, y_pred)
# print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

# Part 2 : Continuous decoding

In this part we will be interested in decoding precise hand movements of a monkey from ECOG data and in order to do so we will procede just as in the Part 1 with
- Pre-processing - where we will see artefact removal
- Feature extraction - by extracting time-frequency features on every electrodes

After this first part we will focus on regression with
- Regression using a linear model 
- Regression using Ridge regularization to prevent overfitting
- Cross validation in order to ensure the robustness of our testing procedure


Finally we will evaluate the model performances using mean square error and correlation.

In [ ]:
# Import packages

import numpy as np # library providing efficient array manipulation
import sklearn # machine learning tools
import matplotlib.pyplot as plt # matlab-like plot library
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

from scipy.signal import iirdesign, sosfilt, convolve, wiener

import tp # load python module written for this project



# Download dataset
dataset_path = 'data/bciciv'
if not tp.dataset_exists(dataset_path):
    tp.download_dataset(dataset_path)
else:
    print(f'dataset already exists in {dataset_path}') 


# Load dataset
dataset = tp.load_data(dataset_path)
fs = 1000

## 2.1 Data pre-processing



In [ ]:
subject = dataset[0]

### Channel selection

It is not uncommon for recordings to have some noisy channels. For example, some electrodes might not properly contact the cortex, or might be faulty.

After visual inspections of the electrodes, we can manually remove them:

In [ ]:
exclude_channels = [54] # list the channels you'd like to remove

n_channels = subject.train_ecog.shape[1]
channel_selection = [i for i in range(n_channels) if i not in exclude_channels]


### Artefact removal 

Some noises are commonly shared between channels, for example line noise (50Hz or 60Hz and its harmonics). A simple method to remove such artefacts is the common average reference (CAR). It consists in computing the average of each channels at a given time step, and substract it from all channels at this time step:

\begin{align}
ECoG(channel,t) = ECoG(channel,t) - \frac{1}{N} \sum_{c=0}^{N-1} ECoG(c,t)
\end{align}

In order to improve the robustness of this method, it is recommended to first exclude channels that may be faulty. It is also possible to use the median instead of the mean, or apply the CAR only on neighbouring channels.


In [ ]:
# Common average reference

ecog_train = tp.common_average_reference(subject.train_ecog[:,channel_selection], axis=1)
ecog_test = tp.common_average_reference(subject.test_ecog[:,channel_selection], axis=1)

In [ ]:
# plot ECoG with CAR vs without

selection = [0]

tp.plot_ecog(subject.train_ecog[1000:2000], selection) # without
tp.plot_ecog(ecog_train[1000:2000], selection) # with


### Feature extraction

In [ ]:
bands = [(1, 10), (10,30), (30,50), (70,200)]

band_filters = tp.compute_band_filters(bands)

# filter
X_train = np.array([sosfilt(filt, ecog_train, axis=0) for filt in band_filters])
X_test = np.array([sosfilt(filt, ecog_test, axis=0) for filt in band_filters])

Y_train = np.array(subject.train_fingers)
Y_test = np.array(subject.test_fingers)

In [ ]:
subject.train_fingers.shape
Y_train.shape

In [ ]:
# plot finger positions

finger = 0 # pick a finger to plot

fig = plt.figure(figsize=(60,15))

ax = fig.subplots(1, 1)

ax = plt.plot(subject.train_fingers[:,finger])
ax = plt.plot(Y_train[:,finger])

In [ ]:
fig = plt.figure(figsize=(50,20))
ax = fig.subplots(nrows=6, ncols=1)

s = slice(1000,2000)

ax[0].plot(X_train[0][s, 0])
ax[1].plot(X_train[1][s, 0])
ax[2].plot(X_train[2][s, 0])
ax[3].plot(X_train[3][s, 0])
ax[4].plot(np.sum(X_train[:,s, 0], axis=0))
ax[5].plot(ecog_train[s,0] - np.sum(X_train[:,s, 0], axis=0))

ax[0].plot(ecog_train[s, 0])
ax[1].plot(ecog_train[s, 0])
ax[2].plot(ecog_train[s, 0])
ax[3].plot(ecog_train[s, 0])
ax[4].plot(ecog_train[s, 0])
ax[5].plot(ecog_train[s, 0])

_ = fig.suptitle('Filtered data (train set)')

In [ ]:
X_train.shape

#### Reshape the ECoG signal 
1. to a shape of (samples, electrodes, frequency bands)
2. to a shape of (samples, features)

In [ ]:
X_train = np.moveaxis(X_train, 0, -1)
X_test = np.moveaxis(X_test, 0, -1)

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

print(X_train.shape)
print(X_test.shape)

#### Compute the ECoG power of each feature
effectively computing it for each frequency band of each electrode

In [ ]:
# make a copy to avoid recomputing everything everytime

X_train_save = np.array(X_train)
X_test_save = np.array(X_test)

In [ ]:
FRAME_LENGTH = 200 # length of the ECoG window on which we compute the signal's power 
HOP = 40 # we downsample the signal to 25Hz, the original frame rate of the glove
BUFFERS = 4 # for each timestep t we concatenate features from timestep t to t-BUFFERS+1 included

X_train = tp.buffering_power(X_train, FRAME_LENGTH, HOP, BUFFERS)
X_test = tp.buffering_power(X_test, FRAME_LENGTH, HOP, BUFFERS)


In [ ]:
# Resample Y to the same frame rate as X: 25 Hz

from scipy.signal import decimate # downsample signals with antialiasing

Y_train = decimate(Y_train, HOP, axis=0, zero_phase=True)
Y_test = decimate(Y_test, HOP, axis=0, zero_phase=True)


In [ ]:
print(X_train.shape)
print(Y_train.shape)

## 2.2 Linear Regression


#### Validation

We're splitting the train set in two to use the first part as train set and the second part as validation set.
Indeed some methods require to optimize parameters and we need some unseen data to evaluate the best values for these parameters.

The test set is left for final evaluation, once the best possible model has been optimized on train and validation.


In [ ]:
X_train_val, X_val, Y_train_val, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=13, shuffle=False)

#### Normalization
We compute mean and standard deviation of the train set and use it to z-score the train, validation and test set. <br>

\begin{align}
zscore(x) = \frac{x-mean(x)}{std(x)}
\end{align}

In [ ]:
# using the sklearn implementation

ecog_scaler = StandardScaler()
X_train_val = ecog_scaler.fit_transform(X_train_val)
X_val = ecog_scaler.transform(X_val)
X_test = ecog_scaler.transform(X_test)

finger_scaler = StandardScaler()
Y_train_val = finger_scaler.fit_transform(Y_train_val)
Y_val = finger_scaler.transform(Y_val)
Y_test = finger_scaler.transform(Y_test)

In [ ]:
from sklearn.linear_model import LinearRegression 

In [ ]:
# optimize lag 
for l in range(-4,5):
    Y_train_lag = tp.lag(Y_train_val, l)
    Y_val_lag = tp.lag(Y_val, l)
    
    reg = LinearRegression()
    reg.fit(X_train_val, Y_train_lag)
    print(f'lag: {l}')
    print(f'\t train: {reg.score(X_train_val, Y_train_lag)}')
    print(f'\t val: {reg.score(X_val, Y_val_lag)}')
    print()

#### Built-in score evaluation

The built-in [score()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge.score) function of sckiti-learn models returns the coefficient of determination of the prediction.

The coefficient of determination $R^2$ is defined as:

\begin{align}
R^2 = (1 - \frac{u}{v})
\end{align}

where $u$ is the residual sum of squares `((y_true - y_pred)** 2).sum()` <br>
and $v$ is the total sum of squares `((y_true - y_true.mean()) ** 2).sum()`

The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). <br>
A constant model that always predicts the expected value of $y$, disregarding the input features, would get
a $R^2$ score of 0.0.

In [ ]:
# settle with 0 lag

for finger in range(Y_train_val.shape[1]):
    reg = LinearRegression()
    reg.fit(X_train_val, Y_train_val[:,finger])
    print(f'Finger {finger}:')
    print(f'\t train: {reg.score(X_train_val, Y_train_val[:,finger])}')
    print(f'\t test:  {reg.score(X_val, Y_val[:,finger])}')


In [ ]:
# Plot predictions (blue) and ground truth (orange)

reg = LinearRegression()
reg.fit(X_train_val, Y_train_val)

fig = plt.figure(figsize=(90,50))
ax = fig.subplots(nrows=10, ncols=1)

Y_train_predict = reg.predict(X_train_val)
Y_test_predict = reg.predict(X_val)

for finger in range(5):
    ax[2*finger].plot(Y_train_predict[:,finger])
    ax[2*finger].plot(Y_train_val[:,finger])
    ax[2*finger].set_ylabel(f'finger {finger+1} (train)')
    
    ax[2*finger+1].plot(Y_test_predict[:,finger])
    ax[2*finger+1].plot(Y_val[:,finger])
    ax[2*finger+1].set_ylabel(f'finger {finger+1} (val)')


#### Decoding speed instead of position

In [ ]:
def derivative(X):
    """ derivative by computing difference between samples n+1 and sample n-1 """
    return (tp.lag(X,1) - tp.lag(X,-1)) / 2

Y_train_speed = derivative(Y_train_val)
Y_val_speed = derivative(Y_val)


fig = plt.figure(figsize=(200,10))
ax = fig.subplots(nrows=1, ncols=1)

# ax.plot(Y_train)
_ = ax.plot(Y_train_speed)


In [ ]:
for finger in range(Y_train.shape[1]):
    reg = LinearRegression()
    reg.fit(X_train_val, Y_train_speed[:,finger])
    print(f'Finger {finger}:')
    print(f'\t train: {reg.score(X_train_val, Y_train_speed[:,finger])}')
    print(f'\t val:  {reg.score(X_val, Y_val_speed[:,finger])}')

In [ ]:
reg = LinearRegression()
reg.fit(X_train_val, Y_train_speed)

fig = plt.figure(figsize=(90,50))
ax = fig.subplots(nrows=10, ncols=1)

Y_train_predict = reg.predict(X_train_val)
Y_val_predict = reg.predict(X_val)

for finger in range(5):
    ax[2*finger].plot(Y_train_predict[:,finger])
    ax[2*finger].plot(Y_train_val[:,finger])
    ax[2*finger].set_ylabel(f'finger {finger+1} (train)')
    
    ax[2*finger+1].plot(Y_val_predict[:,finger])
    ax[2*finger+1].plot(Y_val[:,finger])
    ax[2*finger+1].set_ylabel(f'finger {finger+1} (val)')

## 2.3 Ridge Regression

Ridge regession extends linear regression by adding a regularization factor `alpha` that ensures that the regression coefficients remain close to 0. 

The underlying idea is that higher coefficients may allow better performance on the train set to the cost of being less robust to new data. By keeping the coefficients close to 0, the model should generalize better.

The regression matrix w is computed by minimizing:
\begin{align}
||Y - Xw||^2 + alpha * ||w||^2
\end{align}

The coefficient `alpha` may be optimized on the validation set. It is possible to compute a separate `alpha` value per predicted feature (here for each finger).

In [ ]:
from sklearn.linear_model import Ridge 

In [ ]:
# let's evaluate 1 alpha for all fingers

for i in range(11):
    alpha = i**3
    reg = Ridge(alpha=alpha)
    reg.fit(X_train_val, Y_train_val)
    print(f'alpha={alpha}:')
    print(f'\t train: {reg.score(X_train_val, Y_train_val)}')
    print(f'\t val:  {reg.score(X_val, Y_val)}')

In [ ]:
# now let's optimize a separate alpha for each finger

for finger in range(Y_train.shape[1]):
    print(f'Finger {finger}:')
    for alpha in 0,1,100,1000,10000: # alpha=0 is basically a linear regression
        print(f'\t alpha={alpha}')
        reg = Ridge(alpha=alpha)
        reg.fit(X_train, Y_train[:,finger])
        print(f'\t\t train: {reg.score(X_train, Y_train[:,finger])}')
        print(f'\t\t test:  {reg.score(X_test, Y_test[:,finger])}')


In [ ]:
best_alpha = np.array([1,1,1,1,1]) # Pick the best alpha for each finger

reg = Ridge(alpha=best_alpha)
reg.fit(X_train, Y_train)

fig = plt.figure(figsize=(90,50))
ax = fig.subplots(nrows=10, ncols=1)

Y_train_predict = reg.predict(X_train)
Y_test_predict = reg.predict(X_test)

for finger in range(5):
    ax[2*finger].plot(Y_train_predict[:,finger])
    ax[2*finger].plot(Y_train[:,finger])
    ax[2*finger].set_ylabel(f'finger {finger+1} (train)')
    
    ax[2*finger+1].plot(Y_test_predict[:,finger])
    ax[2*finger+1].plot(Y_test[:,finger])
    ax[2*finger+1].set_ylabel(f'finger {finger+1} (test)')

## 1.3 Evaluation

Let's pick the best model evaluated on the validation set and evaluate it on the test set.

So far we used the sklearn built-in regression metrics because of it's convenience. In practice, different regression problems require different evaluations. 

In [ ]:
# train your best model 

reg = LinearRegression()

reg.fit(X_train_val, Y_train_val)
Y_train_predict = reg.predict(X_train_val)
Y_test_predict = reg.predict(X_test)

print(f'train score: {reg.score(X_train_val,Y_train_val)}')
print(f'test score: {reg.score(X_test,Y_test)}')

### MSE

A very common metric is the mean-squared error, which is defined by:

\begin{align}
mse(x,y) = \frac{1}{n} \sum_{i=0}^{n-1} (x_i - y_i)^2 
\end{align}

The `mse` is derived from the euclidean distance. Due to to the square, it penalizes estimated values that are too far from the ground truth. Minimizing the `mse` is equivalent to minimizing the variance.

Lower is better.

In [ ]:
from sklearn.metrics import mean_squared_error

print(f'train mse: {mean_squared_error(Y_train_predict,Y_train_val)}')
print(f'test mse: {mean_squared_error(Y_test_predict,Y_test)}')

In [ ]:
for finger in range(5):
    print(f'finger {finger}:')
    train_mse = mean_squared_error(Y_train_predict[:,finger],
                                   Y_train_val[:,finger])
    test_mse = mean_squared_error(Y_test_predict[:,finger],
                                  Y_test[:,finger])
    print(f'\ttrain mse: {train_mse}')
    print(f'\ttest mse: {test_mse}')

### Correlation

The Pearson correlation measures the linear relationship between two datasets X and Y. 

The test returns both a correlation coefficient and a p-value. 


A coefficient of 0 means no relationship between both datasets, while a coefficient of 1 or -1 means an exact linear relationship between both datasets.
Positive correlations imply that as x increases, so does y. Negative correlations imply that as X increases, Y decreases.

The p-value indicates the probability of two uncorrelated datasets getting a correlation coefficient as good or better. In practice, it is common to set a threshold for the p-value under which the results are considered significant. Common thresholds are `0.05`, `0.01`, `0.001` with increasing confidence about the results. 



In [ ]:
from scipy.stats import pearsonr

for finger in range(5):
    print(f'finger {finger}:')
    pearson_train = pearsonr(Y_train_predict[:,finger], 
                             Y_train_val[:,finger])
    pearson_test = pearsonr(Y_test_predict[:,finger],
                            Y_test[:,finger])
    
    print(f'\ttrain corr: {pearson_train.statistic:.2f} (p={pearson_train.pvalue:.2E})')
    print(f'\ttest corr: {pearson_test.statistic:.2f} (p={pearson_test.pvalue:.2E})')